In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
import json  
import zipfile
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
df = pd.read_json (r'train.json')
dt = pd.read_json (r'test.json')
print(df)

        bathrooms  bedrooms                       building_id  \
4             1.0         1  8579a0b0d54db803821a35a4a615e97a   
6             1.0         2  b8e75fc949a6cd8225b455648a951712   
9             1.0         2  cd759a988b8f23924b5a2058d5ab2b49   
10            1.5         3  53a5b119ba8f7b61d4e010512e0dfc85   
15            1.0         0  bfb9405149bfff42a92980b594c28234   
...           ...       ...                               ...   
124000        1.0         3  92bbbf38baadfde0576fc496bd41749c   
124002        1.0         2  5565db9b7cba3603834c4aa6f2950960   
124004        1.0         1  67997a128056ee1ed7d046bbb856e3c7   
124008        1.0         2  3c0574a740154806c18bdf1fddd3d966   
124009        1.0         3  d89f514c3ed0abaae52cba7017ac0701   

                    created  \
4       2016-06-16 05:55:27   
6       2016-06-01 05:44:33   
9       2016-06-14 15:19:59   
10      2016-06-24 07:54:24   
15      2016-06-28 03:50:23   
...                     ...   
12

In [4]:
data = df

data = data.drop('building_id', axis=1)
data = data.drop('created', axis=1)
data = data.drop('description', axis=1)
data = data.drop('display_address', axis=1)
data = data.drop('listing_id', axis=1)
data = data.drop('photos', axis=1)
data = data.drop('street_address', axis=1)
data = data.drop('features', axis=1)
data = data.drop('manager_id', axis=1)

In [39]:
test = dt

test = test.drop('building_id', axis=1)
test = test.drop('created', axis=1)
test = test.drop('description', axis=1)
test = test.drop('display_address', axis=1)
test = test.drop('photos', axis=1)
test = test.drop('street_address', axis=1)
test = test.drop('manager_id', axis=1)
test.insert(0,"animal_allow",0)
test.insert(0,"laundry",0)
test.insert(0,"no_fee",0)
test.insert(0,"outdoor",0)
test.features = test.features.astype(str)
test.loc[test['features'].str.contains('Cats Allowed|Dogs Allowed'), 'animal_allow'] = 1
test.loc[test['features'].str.contains('Laundry'), 'laundry'] = 1
test.loc[test['features'].str.contains('No Fee'), 'no_fee'] = 1
test.loc[test['features'].str.contains('Roof Deck|Balcony|Garden/Patio|Terrace|Outdoor Space'), 'outdoor'] = 1
# data.loc[data['features'].str.contains('Doorman|Fitness|Swimming'), 'luxry'] = 1
#data['support'][df.name.str.contains('ball')] = 'ball support'
test = test.drop('features', axis=1)
print(test.head(20))

    outdoor  no_fee  laundry  animal_allow  bathrooms  bedrooms  latitude  \
0         1       0        1             0        1.0         1   40.7185   
1         0       0        0             1        1.0         2   40.7278   
2         0       0        0             1        1.0         0   40.7260   
3         0       0        0             1        1.0         2   40.7321   
5         1       0        1             1        1.0         1   40.7054   
7         1       1        1             1        2.0         2   40.7610   
8         0       0        0             1        3.5         4   40.7358   
11        0       1        1             0        1.0         0   40.7814   
12        0       1        0             0        2.0         4   40.7216   
13        0       0        0             1        1.0         1   40.7553   
14        0       0        0             0        1.0         1   40.7623   
17        1       1        1             0        1.0         1   40.7678   

In [6]:
X_train, X_test , y_train, y_test = train_test_split(data.drop('interest_level', axis=1), data.interest_level, test_size=0.4, random_state=1)
print(X_train)

        bathrooms  bedrooms  latitude  longitude  price
37234         1.0         0   40.6985   -73.9940   2100
63037         1.0         0   40.7602   -73.9689   2675
37184         1.0         3   40.7864   -73.9515   3150
14074         1.0         0   40.7249   -73.9795   1900
119959        1.0         2   40.7874   -73.9738   2995
...           ...       ...       ...        ...    ...
109969        2.0         3   40.7567   -73.9980   5000
81707         1.0         1   40.7878   -73.9799   2895
13102         2.0         4   40.6404   -73.9546   3250
30690         1.0         3   40.7657   -73.9572   3600
82857         1.0         2   40.7568   -73.9982   3000

[29611 rows x 5 columns]


In [66]:
## random forest default

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_proba = clf.predict_proba(test.drop('listing_id', axis=1))
test_data = {'listing_id': test['listing_id'], 'high':y_proba[:,0], 'medium':y_proba[:,2], 'low':y_proba[:,1]}
csvdat = pd.DataFrame(data = test_data)
csvdat.to_csv("test_data de.csv", index = False)
print(csvdat)

        listing_id  high  medium  low
0          7142618  0.00    0.30  0.7
1          7210040  0.20    0.40  0.4
2          7174566  0.00    0.80  0.2
3          7191391  0.35    0.45  0.2
5          7171695  0.00    0.30  0.7
...            ...   ...     ...  ...
124003     6928108  0.10    0.50  0.4
124005     6906674  0.00    0.40  0.6
124006     6897967  0.00    0.00  1.0
124007     6842183  0.10    0.00  0.9
124010     6889319  0.15    0.05  0.8

[74659 rows x 4 columns]


In [8]:
data = df

data = data.drop('building_id', axis=1)
data = data.drop('created', axis=1)
data = data.drop('description', axis=1)
data = data.drop('display_address', axis=1)
data = data.drop('listing_id', axis=1)
data = data.drop('photos', axis=1)
data = data.drop('street_address', axis=1)
data = data.drop('manager_id', axis=1)
data.insert(0,"animal_allow",0)
data.insert(0,"laundry",0)
data.insert(0,"no_fee",0)
data.insert(0,"outdoor",0)
data.features = data.features.astype(str)

In [9]:
data.loc[data['features'].str.contains('Cats Allowed|Dogs Allowed'), 'animal_allow'] = 1
data.loc[data['features'].str.contains('Laundry'), 'laundry'] = 1
data.loc[data['features'].str.contains('No Fee'), 'no_fee'] = 1
data.loc[data['features'].str.contains('Roof Deck|Balcony|Garden/Patio|Terrace|Outdoor Space'), 'outdoor'] = 1
# data.loc[data['features'].str.contains('Doorman|Fitness|Swimming'), 'luxry'] = 1
#data['support'][df.name.str.contains('ball')] = 'ball support'
data = data.drop('features', axis=1)
print(data.head(20))

    outdoor  no_fee  laundry  animal_allow  bathrooms  bedrooms  latitude  \
4         0       0        1             1        1.0         1   40.7108   
6         0       1        1             0        1.0         2   40.7513   
9         0       0        1             0        1.0         2   40.7575   
10        0       0        0             0        1.5         3   40.7145   
15        0       0        1             0        1.0         0   40.7439   
16        0       1        0             0        1.0         3   40.7348   
18        0       1        1             0        2.0         3   40.7302   
19        0       1        1             0        1.0         0   40.7769   
23        0       0        0             0        0.0         1   40.7346   
32        0       0        0             1        3.0         3   40.6990   
33        0       0        1             1        1.0         0   40.7723   
36        1       1        1             1        1.0         2   40.7530   

In [10]:
## random forest default new features
X_train, X_test , y_train, y_test = train_test_split(data.drop('interest_level', axis=1), data.interest_level, test_size=0.4, random_state=1)
print(X_train)
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

        outdoor  no_fee  laundry  animal_allow  bathrooms  bedrooms  latitude  \
37234         0       0        0             1        1.0         0   40.6985   
63037         0       1        1             1        1.0         0   40.7602   
37184         0       0        0             1        1.0         3   40.7864   
14074         0       0        0             1        1.0         0   40.7249   
119959        0       1        1             0        1.0         2   40.7874   
...         ...     ...      ...           ...        ...       ...       ...   
109969        1       1        1             1        2.0         3   40.7567   
81707         0       0        0             1        1.0         1   40.7878   
13102         1       1        1             1        2.0         4   40.6404   
30690         0       0        0             0        1.0         3   40.7657   
82857         1       1        1             0        1.0         2   40.7568   

        longitude  price  


0.7083734359961501

In [11]:
from sklearn.ensemble import AdaBoostClassifier

In [12]:
clf = AdaBoostClassifier(n_estimators=200)
scores = cross_val_score(clf, data.drop('interest_level', axis=1), data.interest_level, cv=5)
scores.mean()

0.7157967240420677

In [13]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\users\jackl\anaconda3\lib\site-packages (0.22.2.post1)
Note: you may need to restart the kernel to use updated packages.


In [67]:
from sklearn.datasets import load_iris
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
X = data.drop('interest_level', axis=1)
y = data.interest_level

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
#    ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42))),
    ('svc',SVC(probability=True)),
    ('rf_2', KNeighborsClassifier(n_neighbors=6)),
    ('nb', GaussianNB()),
    ('adb', AdaBoostClassifier(n_estimators=100, random_state=42)),
    
]
clff = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42
)
clff.fit(X_train, y_train).score(X_test, y_test)

C:\Users\jackl\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7194845193710488

In [68]:
y_proba = clff.predict_proba(test.drop('listing_id', axis=1))
test_data = {'listing_id': test['listing_id'], 'high':y_proba[:,0], 'medium':y_proba[:,2], 'low':y_proba[:,1]}
csvdat = pd.DataFrame(data = test_data)
csvdat.to_csv("test_data2.csv", index = False)
print(csvdat)

        listing_id      high    medium       low
0          7142618  0.042482  0.273814  0.683704
1          7210040  0.097572  0.397847  0.504580
2          7174566  0.087024  0.389507  0.523469
3          7191391  0.218107  0.475581  0.306312
5          7171695  0.011892  0.120436  0.867672
...            ...       ...       ...       ...
124003     6928108  0.142917  0.346891  0.510192
124005     6906674  0.051797  0.296397  0.651806
124006     6897967  0.016354  0.133532  0.850114
124007     6842183  0.007180  0.067008  0.925812
124010     6889319  0.031562  0.182622  0.785816

[74659 rows x 4 columns]


In [71]:
from sklearn.metrics import f1_score
y_pre = clff.predict(X_test)
print(f1_score(y_test, y_pre, average='macro'))
print(f1_score(y_test, y_pre, average='micro'))
print(f1_score(y_test, y_pre, average='weighted'))

0.4879576118718359
0.719484519371049
0.6841541495621957


In [22]:
X = data.drop('interest_level', axis=1)
y = data.interest_level

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42))),
    ('rf_2', KNeighborsClassifier(n_neighbors=5)),  
    ('adb',  AdaBoostClassifier()), 
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42
)
clf.fit(X_train, y_train).score(X_test, y_test)

C:\Users\jackl\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\jackl\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\jackl\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\jackl\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\jackl\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

0.8390338790727833

In [16]:
X = data.drop('interest_level', axis=1)
y = data.interest_level

layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))             
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0.7010050251256281

In [17]:
# General
import numpy as np
 
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
 
# Utilities
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from copy import copy as make_copy
X = data.drop('interest_level', axis=1)
y = data.interest_level
base_clf = [RandomForestClassifier(), LogisticRegression(), 
            AdaBoostClassifier(), #SVC(probability=True),
            KNeighborsClassifier(n_neighbors=5)]
stck_clf = LogisticRegression()

In [18]:
SEED = 2018
for clf in base_clf:
    
    # Set seed
    if 'random_state' in clf.get_params().keys():
        clf.set_params(random_state=SEED)
    
    # Fit model
    clf.fit(X_train, y_train)
    
    # Predict
    y_pred = clf.predict(X_test)
    
    # Calculate accuracy
    acc = accuracy_score(y_test, y_pred)
    print('{} Accuracy: {:.2f}%'.format(clf.__class__.__name__, acc * 100))

RandomForestClassifier Accuracy: 71.52%
LogisticRegression Accuracy: 69.46%
AdaBoostClassifier Accuracy: 71.56%
KNeighborsClassifier Accuracy: 68.60%


In [19]:
clf = StackingClassifier(
    estimators=base_clf, final_estimator=LogisticRegression()
)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [20]:
clf.fit(X_train, y_train).score(X_test, y_test)

TypeError: zip argument #2 must support iteration

In [ ]:
print(f1_score(y_test, y_pre, average='macro'))
print(f1_score(y_test, y_pre, average='micro'))
print(f1_score(y_test, y_pre, average='weighted'))